In [1]:
import numpy as np
import pandas as pd

np.random.seed(0)

In [2]:
def generate_arrival():
    return np.random.exponential(1./10)

def generate_service(i):
    global item_details
    return np.random.normal(item_details[int(i)][0],item_details[int(i)][1],\
                            1)[0]

def generate_cost(i):
    global item_details
    return item_details[int(i)][2]

In [3]:
# Maximum number of "Cashiers" in Coffee shop
max_cashiers = 2

# Maximum number of "Servers" in Coffee shop
max_servers = 3

# Maximum number of items to offer in Coffee shop menu
max_items = 5

# #hr for which this simulation is done
total_time = 8

# "item_details" is list of list. "item_details[i]" is list containing 
#  parameters of distribution which is representative of time taken 
#  to make item "i".

#  I personally believe it reasonable to assume the the random variable  
#  represenative of time taken to make item "i" to follow normal distribution
#  item_details[i][0] = mean time to make item "i"
#  item_details[i][1] = 2nd-central moment for random variable which is
#  represenative of time taken to make item "i"
#  item_details[i][2] = Cost of item "i"
item_details = [[0.01,0.0025,4],[0.02,0.0025,4],[0.03,0.0025,4],[0.04,0.0025,4],\
                [0.05,0.0025,4]]

In [4]:
# simulating arrival events
t_event_arr = 0
t_arrival_list = []
while total_time > 0:
    t_arrival = generate_arrival()
    if total_time >= t_arrival:
        t_event_arr += t_arrival
        total_time -= t_arrival
        t_arrival_list.append(t_event_arr)
    else:
        total_time = 0

In [5]:
# generating order size for the customers randomly
order_size_list = [np.random.randint(1,max_items, size=1)[0] \
                   for i in t_arrival_list]

# generating orders list for the customers randomly
order_list = [np.sort(np.random.randint(0,max_items, size=(1,order_size)))[0]\
             for order_size in order_size_list]

# generating list of unique items ordered by the customers randomly
unique_order_list = [len(np.unique(order_list_row)) for order_list_row \
                    in order_list]

In [6]:
customer_exp = pd.DataFrame({'Time_Of_Arrival':t_arrival_list,\
                             'Order_Size':order_size_list,\
                             'Order_List': order_list, \
                             'Unique_Orders':unique_order_list})

# To trace into account when the order exactly went into queue for given 
# customer
customer_exp[['Cash_Q_Time','Ordering_Time','Cashier_ID']] = \
pd.DataFrame([[0,0,'cashier0']],index=customer_exp.index)

In [7]:
# For identifying the order time
free_time_cashiers = [0]*max_cashiers

customer_index=0
while customer_index < len(t_arrival_list):
    
    if customer_index == 0:
        q_time = 0
        cashier_id = 0
        
    else:
        boolean = 1
        i = 0
        while (boolean):
            if i < max_cashiers:
                if (customer_exp.loc[customer_index,'Time_Of_Arrival']\
                    < free_time_cashiers[i]):
                    i += 1
                    boolean = 1
                    
                else:
                    q_time = 0
                    cashier_id = i
                    boolean = 0
                    
            else:
                cashier_id = np.argmin(free_time_cashiers)
                q_time = np.min(free_time_cashiers) - customer_exp.loc[customer_index,\
                                                              'Time_Of_Arrival']
                boolean = 0
    
    customer_exp.loc[customer_index,'Cash_Q_Time'] = q_time
    customer_exp.loc[customer_index,'Cashier_ID'] = cashier_id
    customer_exp.loc[customer_index,'Ordering_Time'] = \
    (customer_exp.loc[customer_index,'Time_Of_Arrival'] + \
     (customer_exp.loc[customer_index,'Unique_Orders'] * 0.01) + q_time) 
    free_time_cashiers[cashier_id] = customer_exp.loc[customer_index,\
                                                      'Ordering_Time']
    customer_index += 1
    
customer_exp['Ordering_Seq'] = customer_exp['Ordering_Time'].rank()

In [9]:
# Sequencing of customer orders
serving_seq = customer_exp[['Order_List','Ordering_Seq']].copy()

serving_seq_data_manip_counter= 0
if serving_seq_data_manip_counter== 0:
    serving_seq.reset_index(level=0,inplace=True)
    serving_seq_data_manip_counter += 1
else:
    pass

serving_seq.sort_values(by=['Ordering_Seq'],inplace=True)
serving_seq.rename(columns={'index':'Customer_ID'}, inplace=True)

In [10]:
# ref link: https://mikulskibartosz.name/how-to-split-a-list-inside-a-
#           dataframe-cell-into-rows-in-pandas-9849d8ff2401

serving_seq = serving_seq.Order_List.apply(pd.Series).merge(serving_seq,\
                                                            left_index=True,\
                                                            right_index=True)\
.drop(['Order_List'],axis=1)\
.melt(id_vars=['Ordering_Seq','Customer_ID'],value_name='Order')\
.drop("variable", axis=1).dropna().sort_values(by=['Ordering_Seq','Order']).\
reset_index(drop=True)

In [11]:
free_time_servers = [0]*max_servers
serving_seq[['Ordering_Time','Server_ID','Q_time','Order_Build_Time',\
             'Order_End_Time']] = \
pd.DataFrame([[0,0,0,0,0]],index =serving_seq.index)

line_index = 0
while line_index < len(serving_seq):
    
    initial_time = customer_exp.loc[serving_seq.loc[line_index,'Customer_ID'],\
                               'Ordering_Time']
    
    if line_index == 0:

        q_time = 0
        server_id = 0
        
    else:
        boolean = 1
        i = 0
        while (boolean):
            if i < max_servers:
                
                if (initial_time < free_time_servers[i]):
                    i += 1
                    boolean = 1
                    
                else:
                    q_time = 0
                    server_id = i
                    boolean = 0
                    
            else:
                server_id = np.argmin(free_time_servers)
                q_time = np.min(free_time_servers) - initial_time
                boolean = 0
    
    serving_seq.loc[line_index,'Ordering_Time'] = initial_time
    serving_seq.loc[line_index,'Server_ID'] = server_id
    serving_seq.loc[line_index,'Order_Build_Time'] = \
    generate_service(serving_seq.loc[line_index,'Order'])
    serving_seq.loc[line_index,'Revenue'] = \
    generate_cost(serving_seq.loc[line_index,'Order'])
    serving_seq.loc[line_index,'Q_time'] = q_time
    serving_seq.loc[line_index,'Order_End_Time'] =\
    initial_time + q_time + serving_seq.loc[line_index,'Order_Build_Time']
    free_time_servers[server_id] = serving_seq.loc[line_index,'Order_End_Time']
    
    line_index += 1

In [12]:
serving_seq['Q_Time_Before_Customer_Order'] = \
serving_seq.groupby(['Customer_ID'])['Q_time'].transform(min)

serving_seq['Customer_Order_Begin_Time'] = \
serving_seq.groupby(['Customer_ID'])['Order_End_Time'].transform(min)

serving_seq['Revenue_From_Customer'] = \
serving_seq.groupby(['Customer_ID'])['Revenue'].transform(sum)

serving_seq['Customer_Order_End_Time'] = \
serving_seq.groupby(['Customer_ID'])['Order_End_Time'].transform(max)

In [13]:
additional_data_customer_exp = \
serving_seq[['Customer_ID','Q_Time_Before_Customer_Order',\
             'Revenue_From_Customer','Customer_Order_Begin_Time',\
             'Customer_Order_End_Time']].copy()\
.drop_duplicates(subset=['Customer_ID','Q_Time_Before_Customer_Order',\
             'Customer_Order_Begin_Time','Revenue_From_Customer',\
                         'Customer_Order_End_Time'],keep='first')\
.sort_values(by=['Customer_ID']).reset_index(drop=True)

In [17]:
customer_exp = pd.concat([customer_exp,additional_data_customer_exp], axis=1)
# customer_exp['Total_Time_In_Shop'] = \
# customer_exp['Customer_Order_End_Time']-customer_exp['Time_Of_Arrival']

Order
1.0    168.0
2.0    216.0
3.0    160.0
4.0    224.0
Name: Revenue, dtype: float64

In [18]:
# Amount of time Each Cashier was Occupied
customer_exp.groupby(['Cashier_ID'])['Ordering_Time'].sum()

Cashier_ID
0    304.048295
1     64.400702
Name: Ordering_Time, dtype: float64

In [19]:
# Amount of time Each Server was Occupied
serving_seq.groupby(['Server_ID'])['Order_Build_Time'].sum()

Server_ID
0    2.654793
1    2.328163
2    1.876626
Name: Order_Build_Time, dtype: float64

In [20]:
# Revenue as per each item
serving_seq.groupby(['Order'])['Revenue'].sum()

Order
1.0    168.0
2.0    216.0
3.0    160.0
4.0    224.0
Name: Revenue, dtype: float64